In [11]:
import pandas as pd

In [12]:
from catboost import CatBoostClassifier

# 모델 파일 경로 (r을 붙여서 특수문자 이스케이프 방지)
model_path = r'C:\Users\solba\dacon-project\output\catboost_model.cbm'

# 1. 분류 모델일 경우 (Classifier)
model = CatBoostClassifier()
model.load_model(model_path)


print(model.get_params())

{'eval_metric': 'AUC', 'verbose': 200, 'iterations': 1000, 'l2_leaf_reg': 5, 'loss_function': 'Logloss', 'devices': '0', 'task_type': 'GPU', 'depth': 6, 'random_seed': 42, 'learning_rate': 0.03}


In [13]:
test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [14]:
# 필요없는 칼럼 Drop
columns_to_drop = ['generation', 'contest_award', 'completed_semester', 
                   'incumbents_lecture_scale_reason', 'interested_company', 'contest_participation', 'idea_contest']

test = test.drop(columns=columns_to_drop)

In [15]:
# 카테고리 변환
cat_cols = []
for column in test.columns:
    if column == 'ID' or column == 'completed':
        continue
    if column in test.columns:
        test[column] = test[column].astype(str).astype('category')
    
    cat_cols.append(column)

In [16]:
# 결측치 처리: 'Unknown'으로 대체
for column in test.columns:
    if test[column].isnull().sum() > 0:
        test[column].fillna('Unknown', inplace=True)

In [22]:
# 예측 컬럼 추가 및 출력
# ID는 특징에서 제외하고 모델 입력을 맞춥니다.
test_features = test.drop(columns=['ID'], errors='ignore')
proba = model.predict_proba(test_features)[:, 1]
test['completed'] = (proba >= 0.1).astype(int)
print(test['completed'])


0      0
1      0
2      0
3      0
4      0
      ..
809    0
810    0
811    0
812    0
813    0
Name: completed, Length: 814, dtype: int64


In [23]:
test['completed'].unique()

array([0])